In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
mongo_client = MongoClient('18.228.116.164', 27017)
mongo_db = mongo_client['inatel']
mongo_collection = mongo_db['in242']

In [3]:
dados = list(mongo_collection.find())
df = pd.DataFrame(dados)

In [4]:
df.head()

,_id,temperatura,data_coleta
0,5db7839b84a2ef98b84f7ef5,17.748594,2019-10-28 21:11:07.955
1,5db7839c84a2ef98b84f7ef6,27.191536,2019-10-28 21:11:08.956
2,5db7839d84a2ef98b84f7ef7,17.749806,2019-10-28 21:11:09.958
3,5db7839e84a2ef98b84f7ef8,22.026900,2019-10-28 21:11:10.959
4,5db7839f84a2ef98b84f7ef9,19.379516,2019-10-28 21:11:11.960


In [5]:
df.count()

_id            98652
temperatura    98652
data_coleta    98652
dtype: int64

In [6]:
df['temperatura'].mean()

23.485891219527964

In [7]:
df['temperatura'].max()

29.99968034542434

In [8]:
df['temperatura'].min()

17.00002036165729

In [9]:
df = df.drop(columns=['_id'])

In [10]:
def check_temperatura(temperatura):
    if temperatura < 21:
        return 'Frio'
    if temperatura < 26:
        return 'Normal'
    return 'Quente'

In [11]:
check_temperatura(32.5)

'Quente'

In [12]:
df['sensacao'] = df['temperatura'].apply(check_temperatura)

In [13]:
df.tail()

,temperatura,data_coleta,sensacao
35162,22.710910,2019-10-29 22:49:31.565,Normal
35163,26.893494,2019-10-29 22:49:32.566,Quente
35164,20.843331,2019-10-29 22:49:33.567,Frio
35165,23.359086,2019-10-29 22:49:34.569,Normal
35166,21.839947,2019-10-29 22:49:35.570,Normal


In [14]:
df.groupby(['sensacao']).count()

,temperatura,data_coleta
sensacao,,
Frio,10889,10889
Normal,13501,13501
Quente,10777,10777


In [15]:
df.groupby(['sensacao'])['temperatura'].mean()

sensacao
Frio      18.990104
Normal    23.490891
Quente    28.012939
Name: temperatura, dtype: float64

In [16]:
media_min = df.set_index('data_coleta').resample('min')['temperatura'].mean()

In [17]:
media_min.head()

data_coleta
2019-10-28 21:11:00    23.198893
2019-10-28 21:12:00    23.507759
2019-10-28 21:13:00    23.440586
2019-10-28 21:14:00    24.084105
2019-10-28 21:15:00    24.212838
Freq: T, Name: temperatura, dtype: float64

In [18]:
[df['temperatura'].mean() - 2*df['temperatura'].std() ,df['temperatura'].mean() + 2*df['temperatura'].std() ]

[15.956247216627501, 31.00989708602807]

In [19]:
df.set_index('data_coleta', inplace=True)

In [20]:
df.head()

,temperatura,sensacao
data_coleta,,
2019-10-28 21:11:07.955,17.748594,Frio
2019-10-28 21:11:08.956,27.191536,Quente
2019-10-28 21:11:09.958,17.749806,Frio
2019-10-28 21:11:10.959,22.026900,Normal
2019-10-28 21:11:11.960,19.379516,Frio


In [21]:
df.reset_index(inplace=True)

In [22]:
df.tail()

,data_coleta,temperatura,sensacao
35162,2019-10-29 22:49:31.565,22.710910,Normal
35163,2019-10-29 22:49:32.566,26.893494,Quente
35164,2019-10-29 22:49:33.567,20.843331,Frio
35165,2019-10-29 22:49:34.569,23.359086,Normal
35166,2019-10-29 22:49:35.570,21.839947,Normal


In [23]:
# Qual o minuto que tem a media de temperatura mais alta?
q1 = df.set_index('data_coleta').resample('min').mean()

In [24]:
q1.loc[q1.idxmax()['temperatura']]

temperatura    25.226889
Name: 2019-10-29 22:13:00, dtype: float64

In [25]:
# Sinalize todas a medicoes que ultrapassam 28 com alerta true. 

#def ver_alerta(temp):
#     if temp >= 28:
#         return True
#     return False
# df['Alerta'] = df['temperatura'].apply(ver_alerta)

df['Alerta'] = df['temperatura'].apply(lambda x: True if x >= 28 else False )

In [26]:
df

,data_coleta,temperatura,sensacao,Alerta
0,2019-10-28 21:11:07.955,17.748594,Frio,False
1,2019-10-28 21:11:08.956,27.191536,Quente,False
2,2019-10-28 21:11:09.958,17.749806,Frio,False
3,2019-10-28 21:11:10.959,22.026900,Normal,False
4,2019-10-28 21:11:11.960,19.379516,Frio,False
...,...,...,...,...
35162,2019-10-29 22:49:31.565,22.710910,Normal,False
35163,2019-10-29 22:49:32.566,26.893494,Quente,False
35164,2019-10-29 22:49:33.567,20.843331,Frio,False
35165,2019-10-29 22:49:34.569,23.359086,Normal,False


In [27]:
df[df['Alerta'] == True]

,data_coleta,temperatura,sensacao,Alerta
8,2019-10-28 21:11:15.966,29.607994,Quente,True
38,2019-10-28 21:11:46.006,29.116353,Quente,True
47,2019-10-28 21:11:55.019,29.585354,Quente,True
49,2019-10-28 21:11:57.021,28.334223,Quente,True
53,2019-10-28 21:12:01.026,28.417538,Quente,True
...,...,...,...,...
35131,2019-10-29 22:49:00.521,29.537471,Quente,True
35137,2019-10-29 22:49:06.531,28.460230,Quente,True
35149,2019-10-29 22:49:18.546,29.695900,Quente,True
35154,2019-10-29 22:49:23.553,29.605339,Quente,True


In [28]:
# Gerar um DF consolidado com media, min e max e hora.
df2 = pd.DataFrame()

In [29]:
df2['media'] = df.set_index('data_coleta').resample('H')['temperatura'].mean()

In [30]:
df2

,media
data_coleta,
2019-10-28 21:00:00,23.521499
2019-10-28 22:00:00,23.450094
2019-10-28 23:00:00,23.421487
2019-10-29 00:00:00,23.560481
2019-10-29 01:00:00,23.441284
2019-10-29 02:00:00,23.455277
2019-10-29 03:00:00,23.512501
2019-10-29 04:00:00,23.519522
2019-10-29 05:00:00,23.469089


In [31]:
df2['minimo'] = df.set_index('data_coleta').resample('H')['temperatura'].min()

In [32]:
df2

,media,minimo
data_coleta,,
2019-10-28 21:00:00,23.521499,17.001670
2019-10-28 22:00:00,23.450094,17.000266
2019-10-28 23:00:00,23.421487,17.002918
2019-10-29 00:00:00,23.560481,17.005364
2019-10-29 01:00:00,23.441284,17.001720
2019-10-29 02:00:00,23.455277,17.002779
2019-10-29 03:00:00,23.512501,17.000020
2019-10-29 04:00:00,23.519522,17.005252
2019-10-29 05:00:00,23.469089,17.008699


In [33]:
df2['maximo'] = df.set_index('data_coleta').resample('H')['temperatura'].max()

In [34]:
df2

,media,minimo,maximo
data_coleta,,,
2019-10-28 21:00:00,23.521499,17.001670,29.999680
2019-10-28 22:00:00,23.450094,17.000266,29.995169
2019-10-28 23:00:00,23.421487,17.002918,29.997531
2019-10-29 00:00:00,23.560481,17.005364,29.998907
2019-10-29 01:00:00,23.441284,17.001720,29.992742
2019-10-29 02:00:00,23.455277,17.002779,29.996982
2019-10-29 03:00:00,23.512501,17.000020,29.999112
2019-10-29 04:00:00,23.519522,17.005252,29.999503
2019-10-29 05:00:00,23.469089,17.008699,29.995053


In [35]:
# outra possibilidade de resolucao
df2 = df.set_index('data_coleta').resample('H')['temperatura'].agg(['mean','min','max'])

In [36]:
df2

,mean,min,max
data_coleta,,,
2019-10-28 21:00:00,23.521499,17.001670,29.999680
2019-10-28 22:00:00,23.450094,17.000266,29.995169
2019-10-28 23:00:00,23.421487,17.002918,29.997531
2019-10-29 00:00:00,23.560481,17.005364,29.998907
2019-10-29 01:00:00,23.441284,17.001720,29.992742
2019-10-29 02:00:00,23.455277,17.002779,29.996982
2019-10-29 03:00:00,23.512501,17.000020,29.999112
2019-10-29 04:00:00,23.519522,17.005252,29.999503
2019-10-29 05:00:00,23.469089,17.008699,29.995053


In [37]:
# atribuir um valor NaN (Not a Number) em uma coluna
# df2['soma'] = pd.np.nan

In [38]:
# reordenar
df2 = df2[['max','min','mean']]

In [39]:
df2

,max,min,mean
data_coleta,,,
2019-10-28 21:00:00,29.999680,17.001670,23.521499
2019-10-28 22:00:00,29.995169,17.000266,23.450094
2019-10-28 23:00:00,29.997531,17.002918,23.421487
2019-10-29 00:00:00,29.998907,17.005364,23.560481
2019-10-29 01:00:00,29.992742,17.001720,23.441284
2019-10-29 02:00:00,29.996982,17.002779,23.455277
2019-10-29 03:00:00,29.999112,17.000020,23.512501
2019-10-29 04:00:00,29.999503,17.005252,23.519522
2019-10-29 05:00:00,29.995053,17.008699,23.469089


In [40]:
df2.to_excel('consolidado.xls')